<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/34texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
Rafa Caballero


<table>
    <tr>
      <td>Análisis de textos
      </td>
      <td>
      <img src="https://www.libreriadelprado.com/3720-large_default/la-mejor-cocinera-recetas-de-cocina.jpg"  width=150/>
      </td>
     </tr>
</table>






### Carga de bibliotecas y modelos de spaCy

In [ ]:
modules = ["bs4", "spacy", "pandas","seaborn","scipy","pysentimiento",
           "accelerate==0.20.1"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")
          try:
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

  print("¡Terminado!")

instala(modules)

# ahora el modelo para español; ojo esto puede tardar y solo debe hacerse una vez, comentarlo tras la primera ejecución
# tras ejecutarlo reiniciar (Entorno de Ejecución + Reiniciar entorno de ejecución)
!python -m spacy download es_core_news_md

In [ ]:
# cargamos el modelo
#import es_core_news_md
import spacy

nlp = spacy.load('es_core_news_md') #es_core_news_md.load()

stops= ["gramo", "gramos","kilo","kilos","litro", "litros","punto","hora","horas","minuto","minutos","fuego",
        "fuente","cacerola","horno",
       "cucharada","cucharadas","trozo","trozos","molde","moldes"]
[nlp.Defaults.stop_words.add(w) for w in stops]
print("Listo!")

 Cargamos el texto de la página:

https://www.gutenberg.org/cache/epub/8870/pg8870.html  mediante requests (para obtener) y BeautifulSoup (para analizar y extraer el texto) dejándolo en una variable de nombre `texto`. Nos aseguramos de que `requests` no da error, mostrando un mensaje si el `status_code` es distinto de 200

In [ ]:
from bs4 import BeautifulSoup
import requests
pagina = "https://www.gutenberg.org/cache/epub/8870/pg8870.html"

page = requests.get(pagina)
if page.status_code==200:
    soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
    texto = soup.text
else:
    print("Error ", page.status_code)

 Aplicamos el modelo `nlp`al `texto` dejando el resultado en una variable `doc`.
A partir de `doc` extraer una lista de todos los lexemas en mayúsculas de los tokens que

1) Sean valores alfauméricos
2) No correspondan a palabras vacías
3) Sean sustantivos

El resultado será una lista de palabras en mayúscula (lexemas) que guardaremos en una variable `palabras`

Ayuda: un token es un sustantivo si `token.pos_ == "NOUN"`. La palabra *pos* viene de *part of speech* e indica el tipo de palabra que es. Ver la [ayuda de spaCy](https://spacy.io/usage/linguistic-features) para un ejemplo

In [ ]:
doc = nlp(texto)
palabras = [p.lemma_.upper() for p in doc
            if p.is_alpha and not p.is_stop and p.pos_=="NOUN"]
palabras

Nos quedamos solo con las `N` palabras más frecuentes en `palabras` mediante el método `Counter`. El resultado sera una lista `frecuentes`.

In [ ]:
from collections import Counter
N=25

c = Counter(palabras)
frecuentes = [p for p,_ in c.most_common(N)]
frecuentes

A partir de `doc` obtener una lista `frases`con todas las oraciones del libro en mayúsculas. El resultado esperado será de la forma:

        ['\n',
         'THE PROJECT GUTENBERG EBOOK OF LA MEJOR COCINERA, RECETAS DE COCINA, BY CALLEJA\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTHE PROJECT GUTENBERG EBOOK OF LA MEJOR COCINERA...]

(recordar que .text devuelve el texto asociado, ya sea a una palabra, una oración, etc.)

In [ ]:
frases = [o.text.upper() for o in doc.sents]

# para probar
frases[98:100]

Ahora escribimos una función:

Nombre `palabrasEnFrase`

Parámetros:

    palabras: una lista de strings representando palabras
    frase: un string representando una frase
   
Salida: una lista de 0s y 1s, con la misma longitud que `palabras` tal que en la posición i tendrá un 1 si la palabra iésima aparece en la frase y 0 en otro caso



In [ ]:
def palabrasEnFrase(palabras,frase):
    # solución
    return [ 0 if p not in frase else 1 for p in palabras]

# salida esperada: [1, 0, 1, 0, 1, 0]
palabrasEnFrase(["PRINCESA","RAMA", "RANA", "PRINCIPE","POZO", "GOZO"],
                "HABÍA UNA VEZ UNA LINDA RANA QUE AL CAERSE A UN POZO SE CONVIRTIÓ EN UNA HORRIBLE PRINCESA")

A partir de `frecuentes`(ejercicio 3) y `frases` (ejercicio 4) queremos obtener una lista `pertenencia`donde

1) `pertenencia` tiene tantos elementos como `frases` (uno por frase)
2) El elemento i-ésimo de `pertenencia` es una lista de  1s y 0s con la misma longitud que `frecuentes` indicando si cada palabra de `frecuentes` está en la frase i-ésima.

Con otras palabras pertenencia es una lista de listas donde `pertenencia[i][j] == 1` si `frecuentes[j]` aparece en `frases[i]`, y 0 en otro caso

Nota: conviene usar la función del ejercicio 5

In [ ]:
pertenencia = [palabrasEnFrase(frecuentes,f) for f in frases]
pertenencia

In [ ]:
# para probar;
import pandas as pd
df = pd.DataFrame(pertenencia,columns=frecuentes)
df

El siguiente código elimina aquellas filas que no tienen ninguna de las palabras más frecuentes

In [ ]:
df2 = df[df!=0].dropna(how="all").fillna(0).reset_index(drop=True)
df2

Mostrar las parejas de columnas diferentes en `df2`que tengan una correlación superior a 0.25 en valor absoluto

Ayuda: Si llamamos correlaciones al dataframe generado por corr, podemos acceder a la correlación de la columna c1 con la c2 (con c1,c2 nombres de columnas de df2) mediante `correlaciones.loc[c1,c2]`



In [ ]:
correlaciones = df2.corr()
for c1 in df2.columns:
    for c2 in df2.columns:
        if c1<c2:
            v = correlaciones.loc[c1,c2]
            if abs(v)>0.20:
                print(c1,c2,v)

¿Qué dos ingredientes parece más improbable encontrar juntos a la vista de estas correlaciones?

Solución:

Análisis de sentimiento



In [ ]:
#!pip install accelerate -U

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pysentimiento import create_analyzer





analizador_py_sent = create_analyzer(task="sentiment", lang="es")
#analizador_py_odio = create_analyzer(task="hate_speech", lang="es")
analizador_py_emo = create_analyzer(task="emotion", lang="es")





In [ ]:
msjs_tor = ["Me encanta la tortilla de patata con cebolla",
            "No puedo creer que haya gente que tome la tortilla con cebolla",
            "sincebollista hasta la muerte, abajo la cebolla",
            "La cebolla no se toca",
            "Ojalá desaparezcan del mundo todos los cebollistas"]
msjs_vacas = ["¡Qué caro está todo!",
              "Espero que mi equipo gane la liga este año",
              "Qué pena que el servicio al cliente sea tan lento",
              "No me ha gustado nada el producto, pésima relación calidad/precio",
              "Recomendaría este hotel, lo mejor la limpieza",
              "Olvídame fuerte, igual que te amé"]
msjs = msjs_tor + msjs_vacas

for frase in msjs:
    analisis_sent = analizador_py_sent.predict(frase)
    analisis_emo = analizador_py_emo.predict(frase)
    print(frase)
    print("Sentimiento", analisis_sent)
    print("Emoción",analisis_emo)
    print("="*30)